In [1]:
import pandas as pd, numpy as np, scipy, os, itertools
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy
from func import loaders

In [2]:
outcome_df = pd.read_csv('./data/clinical data/clinical_data_2.csv')[['subj', 'AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]']]
data_type ='mean' #mean, var
ld_excel = loaders(data_type = data_type)
ld_mat = loaders(data_type=data_type)
raw_data_path = './data/dFC matrix matlab/'
for next_path in sorted(os.listdir(raw_data_path)):
    if 'C0018FC_z.mat' not in next_path:
        ld_mat.load_mat(data_path = os.path.join(raw_data_path,next_path))
alff_index, alff_value, alff_columns = ld_excel.load_csv(data_path=f'./data/mdALFF matrix excel/mdALFF_{data_type}.csv')
fc_value = ld_mat.callback()

In [3]:
from import_library import *

def data_spilt(patient_value, label_, outcome_name = 'AD8主觀認知障礙[0-1,>=2'):
    if 'MOCA' in outcome_name:
        label_s = [1 if (i>=24)  else 0  for i in list(label_)]
    elif 'AD8' in outcome_name:
        label_s = [1 if i>=2 else 0 for i in list(label_)]
        label_ = label_+1
    X_train, X_test, Y_label, _  = train_test_split(patient_value, list(label_s) , stratify=list(label_s), random_state=123)
    y_train, y_test, _, _ = train_test_split(list(label_), list(label_s) , stratify=list(label_s), random_state=123)
    return X_train, y_train, X_test, y_test
    # 1.feature selection for total dataset or train dataset
    # 2.feature selection overfitting?[boruta]. depth?
    # 3._add_shadows_get_imp

model_stack_ALFF = [None, None]
model_stack_FC = [None, None]
model_feature_stack = []
for idx, outcome_name in enumerate(['AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]']):
# for idx, outcome_name in enumerate(['AD8主觀認知障礙[0-1,>=2]']):
    forest = RandomForestRegressor(max_depth = 5, n_jobs=-1, 
                        random_state=123, criterion = 'absolute_error')
    # FC feature selection
    print(f"FC {data_type} Feature Selection:", outcome_name)
    X, y, _, _ = data_spilt(fc_value, outcome_df[outcome_name], outcome_name,)
    feat_selector_fc = BorutaPy(forest, verbose=0, random_state=123, max_iter=10)
    feat_selector_fc.fit(np.array(X), np.array(y))
    model_stack_FC[idx] = (feat_selector_fc)
    # feature_df = pd.DataFrame([f'FC_{data_type}_{i}' for i in range (fc_value.shape[1])], columns=['features'])
    # feature_df['rank'] = feat_selector_fc.ranking_
    # feature_df = feature_df.sort_values(by=['rank']).reset_index()
    # feature_df.to_csv(f'./feature_selection/{outcome_name}_FC_{data_type}_feature-selection.csv')
    ranking_stack = []
    for (value, colum) in zip(feat_selector_fc.ranking_, [f'FC_{data_type}_{i}' for i in range (fc_value.shape[1])]):
        ranking_stack.append([value, colum])
    # ranking_stack = sorted(ranking_stack)
    pd_rs = pd.DataFrame(ranking_stack, columns=['rank', 'feature_location'])
    pd_rs.to_csv(f'./feature_selection_result/{outcome_name}_{data_type}_fc.csv', index=False)
    model_feature_stack.append(pd_rs)

    # ALFF feature selection
    print(f"ALFF {data_type} Feature Selection:", outcome_name)
    X, y, _, _  = data_spilt(alff_value, outcome_df[outcome_name], outcome_name = outcome_name)
    feat_selector_alff = BorutaPy(forest, verbose=0, random_state=123, max_iter=10, alpha=0.01, two_step=True)
    feat_selector_alff.fit(np.array(X), np.array(y))
    model_stack_ALFF[idx] = feat_selector_alff
    # feature_df = pd.DataFrame(alff_columns.tolist(), columns=['features'])
    # feature_df['rank'] = feat_selector_alff.ranking_
    # feature_df = feature_df.sort_values(by=['rank']).reset_index()
    # feature_df.to_csv(f'./feature_selection/{outcome_name}_ALFF_{data_type}_feature-selection.csv')
    ranking_stack = []
    for (value, colum) in zip(feat_selector_alff.ranking_,X.columns.tolist()):
        ranking_stack.append([value, colum])
    ranking_stack = sorted(ranking_stack)
    pd_rs = pd.DataFrame(ranking_stack, columns=['rank', 'feature_location'])
    pd_rs.to_csv(f'./feature_selection_result/{outcome_name}_{data_type}_RFabsolute_error.csv', index=False)
    model_feature_stack.append(pd_rs)

ALFF mean Feature Selection: AD8主觀認知障礙[0-1,>=2]
ALFF mean Feature Selection: MOCA客觀認知測驗分數[<=23,>=24]


In [4]:
for i in zip(model_feature_stack, ['AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]']):
    print(i[1])
    print(i[0].head(10))
    print('\n')

AD8主觀認知障礙[0-1,>=2]
   rank      feature_location
0     2    288_FrontoParietal
1     3  211_VentralAttention
2     4  13_SomatomotorDorsal
3     5    285_FrontoParietal
4     6          248_Salience
5     7   65_CinguloOpercular
6     8       279_DefaultMode
7     9       277_DefaultMode
8    10  18_SomatomotorDorsal
9    11            143_Visual


MOCA客觀認知測驗分數[<=23,>=24]
   rank        feature_location
0     2     60_CinguloOpercular
1     3  255_SomatomotorLateral
2     4            205_Salience
3     5            6_unassigned
4     6         129_DefaultMode
5     7         111_DefaultMode
6     8              143_Visual
7     9              240_Reward
8    10             77_Auditory
9    11          86_DefaultMode




In [5]:
# for idx, i in enumerate(model_feature_stack):
#     rank_sorted = i['rank'].tolist()
#     rank_value = [max(rank_sorted)-i for i in rank_sorted]
#     model_feature_stack[idx]['rank_value'] = rank_value

In [6]:
# import matplotlib.pyplot as plt
# plt_title = [f"FC {data_type} Feature Selection: AD8", f"ALFF {data_type} Feature Selection: AD8", f"FC {data_type} Feature Selection: MOCA", f"ALFF {data_type} Feature Selection: MOCA"]
# for idx, i in enumerate(plt_title):
#     fig = plt.figure(figsize=(40,5))
#     plt.title(i) # 圖的標題
#     plt.xlabel("x axis", fontsize=10) # x軸的名稱

#     plt.ylabel("y axis", fontsize=10) # y軸的名稱
#     plt.bar( model_feature_stack[idx]['feature_location'].tolist()[:100], model_feature_stack[idx]['rank_value'].head(100).values) # 繪製長條圖
#     plt.xticks(rotation=90)
#     plt.show() # 顯現圖形

In [7]:
# new_df = pd.DataFrame(X.columns.tolist(), columns =['feature_location'])

In [8]:
# print(model_stack_ALFF[0].importance_history_[8])

In [9]:
# for model_, d_t in zip(model_stack_FC['BorutaPy'],[f'FC {data_type} - AD8', f'FC {data_type} - MOCA']):
#     print('-----BorutaPy ',d_t,'-----')
#     X, y, _, _ = data_spilt(fc_value, outcome_df[outcome_name], outcome_name,)
#     X = pd.DataFrame(X, columns=[f'FC_{data_type}_{i}' for i in range (fc_value.shape[1])])
#     green_area = X.columns[model_.support_].to_list()
#     blue_area  = X.columns[model_.support_weak_].to_list()
#     print('features in the green area:', green_area)
#     print('features in the blue area:', blue_area)

In [10]:
# for model_, d_t in zip(model_stack_ALFF['BorutaPy'],[f'ALFF {data_type} - AD8', f'ALFF {data_type} - MOCA']):
#     print('-----BorutaPy ',d_t,'-----')
#     X, y, _, _  = data_spilt(alff_value, outcome_df[outcome_name], outcome_name = outcome_name)
#     green_area = X.columns[model_.support_].to_list()
#     blue_area  = X.columns[model_.support_weak_].to_list()
#     print('features in the green area:', green_area)
#     print('features in the blue area:', blue_area)